In [ ]:
import json
import os

def extract_file_name(obj):
    global file_names
    if 'file_name' in obj:  # Check if the object has a 'file_name' key
        file_names.append(obj['file_name'])  # Extract the value of that key

def get_object(objs):
    global objects_list
    for obj in objs:
        extract_file_name(obj)  # Call the function to extract the file name
        if isinstance(obj, dict):  # Check if the object is a dictionary (JSON object)
            objects_list.append(obj)

# Initialize lists to store file names and objects
file_names = []
objects_list = []

# Specify the path to the JSON file relative to the current working directory
file_path = os.path.join('..', 'upload', 'assets.json')

try:
    data = json.load(open(file_path))
except FileNotFoundError as e:
    print(f"File not found: {e}")
else:
    get_object(data)  # Call the function to get all objects

print("File Names:", file_names)
print("JSON Objects:", objects_list)

In [ ]:
import json
import os
from ipfs_functions import upload_json_to_ipfs, download_json_from_ipfs, upload_file_to_ipfs

def extract_file_name(obj):
    global file_names
    if 'file_name' in obj:  # Check if the object has a 'file_name' key
        file_names.append(obj['file_name'])  # Extract the value of that key

def get_object(objs):
    global objects_list
    for obj in objs:
        extract_file_name(obj)  # Call the function to extract the file name
        if isinstance(obj, dict):  # Check if the object is a dictionary (JSON object)
            objects_list.append(obj)

# Initialize lists to store file names and objects
file_names = []
objects_list = []

# Specify the path to the JSON file relative to the current working directory
file_path = os.path.join('upload', 'assets.json')

try:
    data = json.load(open(file_path))
except FileNotFoundError as e:
    print(f"File not found: {e}")
else:
    get_object(data)  # Call the function to get all objects

# Iterate over file names and objects
for i, obj in enumerate(objects_list):
    print(f"File Name: {file_names[i]}")
    upload_file_to_ipfs("upload/"+file_names[i])

    print()  # Print a newline for readability

    print(obj)
    upload_json_to_ipfs(obj)
    # download_json_from_ipfs(obj)
    print()  # Print a newline for readability



In [ ]:
import json
import os
from ipfs_functions import upload_json_to_ipfs, download_json_from_ipfs, upload_file_to_ipfs

def extract_file_names(data):
    """
    Extract file names from a JSON object or list of objects.
    
    Args:
        data (dict or list): The input JSON data
    
    Returns:
        list: A list of file names
    """
    file_names = []
    for obj in data:
        if isinstance(obj, dict) and 'file_name' in obj:
            file_names.append(obj['file_name'])
    return file_names

def process_objects(data):
    """
    Process a JSON object or list of objects by extracting file names,
    uploading files to IPFS, and uploading JSON objects to IPFS.
    
    Args:
        data (dict or list): The input JSON data
    """
    file_names = extract_file_names(data)
    for obj in data:
        if isinstance(obj, dict):
            print(f"File Name: {file_names[0]}")
            upload_file_to_ipfs("upload/"+file_names[0])
            print()
            print(obj)
            upload_json_to_ipfs(obj)
            # download_json_from_ipfs(obj)

def main():
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        process_objects(data)

if __name__ == '__main__':
    main()

In [ ]:
import json
import os
from ipfs_functions import upload_json_to_ipfs, download_json_from_ipfs, upload_file_to_ipfs

def extract_file_names(data):
    """
    Extract file names from a JSON object or list of objects.
    
    Args:
        data (dict or list): The input JSON data
    
    Returns:
        list: A list of file names
    """
    file_names = []
    for obj in data:
        if isinstance(obj, dict) and 'file_name' in obj:
            file_names.append(obj['file_name'])
    return file_names

def process_objects(data):
    """
    Process a JSON object or list of objects by extracting file names,
    uploading files to IPFS, and uploading JSON objects to IPFS.
    
    Args:
        data (dict or list): The input JSON data
    """
    file_names = extract_file_names(data)
    for i, obj in enumerate(data):
        if isinstance(obj, dict):
            print(f"\nProcessing Object {i+1}:")
            print(f"File Name: {file_names[i]}")
            upload_file_to_ipfs("upload/"+file_names[i])
            print()
            print("JSON Object:")
            print(json.dumps(obj, indent=4))  # Pretty-print JSON object
            upload_json_to_ipfs(obj)
            # download_json_from_ipfs(obj)

def main():
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        process_objects(data)

if __name__ == '__main__':
    main()

In [ ]:
import json
import os
from ipfshttpclient import connect
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def upload_file_to_ipfs(client, file_path):
    """
    Upload a file to IPFS.

    Args:
        client (ipfshttpclient): IPFS client connection
        file_path (str): Path to the file to upload

    Returns:
        str: Hash of the uploaded file
    """
    result = client.add(file_path)
    logger.info(f"Uploaded file {result['Name']} with hash {result['Hash']}")
    return result['Hash']

def upload_json_to_ipfs(client, json_data):
    """
    Upload JSON data to IPFS.

    Args:
        client (ipfshttpclient): IPFS client connection
        json_data (dict): JSON data to upload

    Returns:
        str: CID of the uploaded JSON data
    """
    key = client.add_json(json_data)
    logger.info(f"Uploaded JSON with metadata CID {key}")
    return key

def download_file_from_ipfs(client, cid, output_path):
    """
    Download a file from IPFS.

    Args:
        client (ipfshttpclient): IPFS client connection
        cid (str): CID of the file to download
        output_path (str): Path where the downloaded file will be saved

    Returns:
        None
    """
    file_data = client.cat(cid)
    with open(output_path, 'wb') as f:
        f.write(file_data)

def process_json_data(client, data):
    """
    Process JSON data by uploading files and JSON to IPFS.

    Args:
        client (ipfshttpclient): IPFS client connection
        data (dict or list): JSON data to process

    Returns:
        None
    """
    file_names = []
    for obj in data:
        if isinstance(obj, dict) and 'file_name' in obj:
            file_names.append(obj['file_name'])

    for i, obj in enumerate(data):
        if isinstance(obj, dict):
            logger.info(f"Processing object {i+1}")
            upload_file_to_ipfs(client, "upload/" + file_names[i])
            logger.info("JSON object:")
            logger.info(json.dumps(obj, indent=4))
            upload_json_to_ipfs(client, obj)
            # download_file_from_ipfs(client, obj['cid'], 'output/' + file_names[i])

def main():
    client = connect('/dns/10.0.0.100/tcp/5001/http')  # Replace with your IP address and port
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        process_json_data(client, data)
        

if __name__ == '__main__':
    main()

In [ ]:
from json_ipfs_functions import *


def main():
    # client = connect('/dns/10.0.0.100/tcp/5001/http')  # Replace with your IP address and port
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        results = process_json_data(data)
        # Print the file CIDs and JSON metadata CIDs.
        for fn, cid in results["file_cids"].items():
            print(f"File CID: {cid} (for file {fn})")
            # file_cid = cid,fn
            # print (file_cid)
        for key, json_cid in results["json_cids"].items():
            # metadata_cid = {json_cid}
            print(f"JSON CID: {json_cid} (for file {key.split('_')[0]})")
        # hash = set_account_detail(address, account, file_cid, metadata_cid)

if __name__ == '__main__':
    main()

In [ ]:
from json_ipfs_functions import *

def process_json_data(data):
    """
    Process JSON data by uploading files and JSON to IPFS.

    Args:
        client (ipfshttpclient): IPFS client connection
        data (dict or list): JSON data to process

    Returns:
        dict: Dictionary with file CIDs and JSON metadata CIDs.
    """
    file_names = []
    json_cids = {}
    for obj in data:
        if isinstance(obj, dict) and 'file_name' in obj:
            file_names.append(obj['file_name'])

    results = {}
    for i, obj in enumerate(data):
        if isinstance(obj, dict):
            logger.info(f"Processing object {i+1}")
            cid_file = upload_file_to_ipfs("upload/" + file_names[i])
            key_json = upload_json_to_ipfs(obj)
            json_cids[f"{obj['file_name']}_json_cid"] = key_json
            results[f"{file_names[i]}_cid"] = cid_file  # Store file CID with its name
            logger.info("JSON object:")
            logger.info(json.dumps(obj, indent=4))

    # Display the file CIDs and JSON metadata CIDs as key/value pairs.
    logger.info("\nFile CIDs:")
    for k, v in results.items():
        logger.info(f"{k}: {v}")
    
    logger.info("\nJSON Metadata CIDs:")
    for k, v in json_cids.items():
        logger.info(f"{k}: {v}")

    # Return a dictionary with the file CIDs and JSON metadata CIDs.
    return {"file_cids": results,
            "json_cids": json_cids}

def main():
    # client = connect('/dns/10.0.0.100/tcp/5001/http')  # Replace with your IP address and port
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        results = process_json_data(data)
        
        # Print the file CIDs and JSON metadata CIDs.
        logger.info("\nResults:")
        for k, v in results.items():
            if isinstance(v, dict):
                logger.info(f"{k}:")
                for sub_k, sub_v in v.items():
                    logger.info(f"  {sub_k}: {sub_v}")
            else:
                logger.info(f"{k}: {v}")

if __name__ == '__main__':
    main()
